Первое задание

Загружаем тексты

In [1]:
import gzip

from dataclasses import dataclass
from typing import Iterator

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("data/news.txt.gz"))


In [2]:
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, gte, lte, is_capitalized
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.relations import gnc_relation

ENTRY = fact("Entry", ["full_name", "birthday", "birth_place"])
NAME = fact("Name", ["first", "last", "patronimic"])
BIRTHDAY = fact("date", ["d", "m", "y"])
BIRTH_PLACE = fact("Birth_place", ["place"])


DAY = and_(gte(1), lte(31)) # >= 1 и <=31
MONTH = morph_pipeline(["Январь", "Февраль", "Март", "Апрель", "Май", "Июнь", "Июль", "Август", "Сентябрь", "Октябрь", "Ноябрь", "Декабрь"])
YEAR = and_(gte(1), lte(2024)) # >= 1 и <= 2024

# Немного учтем возможность разного порядка написания ФИО

name_surn_patr = rule(
    gram("Name").interpretation(NAME.first.inflected()),
    gram("Surn").interpretation(NAME.last.inflected()),
    gram("Patr").interpretation(NAME.patronimic.inflected()).optional()
).interpretation(NAME)

surn_name_patr = rule(
    gram("Surn").interpretation(NAME.last.inflected()),
    gram("Name").interpretation(NAME.first.inflected()),
    gram("Patr").interpretation(NAME.patronimic.inflected()).optional()
).interpretation(NAME)

birthday_words = rule(
    DAY.interpretation(BIRTHDAY.d).optional(),
    MONTH.interpretation(BIRTHDAY.m).optional(),
    morph_pipeline(["в"]).optional(),
    YEAR.interpretation(BIRTHDAY.y).optional(),
    morph_pipeline(["Году"]).optional()
).interpretation(BIRTHDAY).optional()

birthday_nums = rule(
    DAY.interpretation(BIRTHDAY.d).optional(),
    '.',
    and_(gte(1), lte(12)).interpretation(BIRTHDAY.m).optional(),
    '.',
    YEAR.interpretation(BIRTHDAY.y).optional(),
    morph_pipeline(["Года"]).optional()
).interpretation(BIRTHDAY).optional()

birth_place = rule(
    morph_pipeline(["в"]),
    is_capitalized().interpretation(BIRTH_PLACE.place).optional()
).interpretation(BIRTH_PLACE).optional()

entry = rule(
    or_(
    name_surn_patr.interpretation(ENTRY.full_name),
    surn_name_patr.interpretation(ENTRY.full_name)
    ),
    morph_pipeline(["Родился", "был рожден", "появился на свет"]),

    # Так как могут идти в любом порядке

    or_(
        or_(
            birthday_nums.interpretation(ENTRY.birthday),
            birthday_words.interpretation(ENTRY.birthday),
        ),
        birth_place.interpretation(ENTRY.birth_place)
    ),
    or_(
        or_(
             birthday_nums.interpretation(ENTRY.birthday),
            birthday_words.interpretation(ENTRY.birthday),
        ),
        birth_place.interpretation(ENTRY.birth_place)
    )
).interpretation(ENTRY)

parser = Parser(entry)


In [3]:
text = "Иванов Иван Иванович родился 18.01.1978 в Белграде"

for match in parser.findall(text):
    print(match.fact)

Entry(full_name=Name(first='иван', last='иванов', patronimic='иванович'), birthday=date(d='18', m='01', y='1978'), birth_place=Birth_place(place='Белграде'))


In [4]:
text = "Иванов Иван Иванович родился 18 января 1978 года в Белграде"

for match in parser.findall(text):
    print(match.fact)

Entry(full_name=Name(first='иван', last='иванов', patronimic='иванович'), birthday=date(d='18', m='января', y='1978'), birth_place=Birth_place(place='Белграде'))


In [3]:
from tqdm import tqdm 

for text in tqdm(texts, disable=False):
    try:
        for match in parser.findall(text.text):
         print(match.fact)
    except:
       # empty
       pass

  0%|          | 0/10000 [00:00<?, ?it/s]

  4%|▍         | 436/10000 [00:18<06:29, 24.58it/s]

Entry(full_name=Name(first='андрей', last='курносенко', patronimic=None), birthday=None, birth_place=Birth_place(place='Севастополе'))


  5%|▌         | 544/10000 [00:23<05:44, 27.47it/s]

Entry(full_name=Name(first='иосиф', last='кобзон', patronimic=None), birthday=date(d=None, m=None, y=None), birth_place=None)


...

 98%|█████████▊| 9791/10000 [07:55<00:09, 22.18it/s]

Entry(full_name=Name(first='в', last='брак', patronimic=None), birthday=None, birth_place=None)


 99%|█████████▊| 9862/10000 [07:59<00:07, 18.07it/s]

Entry(full_name=Name(first='алексей', last='ремизов', patronimic=None), birthday=date(d=None, m=None, y='1877'), birth_place=Birth_place(place='Москве'))


100%|██████████| 10000/10000 [08:05<00:00, 20.58it/s]
